Inception

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 1. 資料前處理（299x299 for Inception） -----------
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=inception_transform)
test_dataset = datasets.ImageFolder(test_path, transform=inception_transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 2. 模型定義：Inception-v3 Transfer -----------
class InceptionV3Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.base = models.inception_v3(pretrained=True, aux_logits=True)
        for param in self.base.parameters():
            param.requires_grad = False

        self.base.fc = nn.Identity()  # 去掉原本分類層

        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.base(x)
        if isinstance(x, tuple):  # 有 aux_logits 時會回傳 InceptionOutputs
            x = x.logits
        return self.classifier(x)


# ----------- 3. 損失計算與 EarlyStopping -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# ----------- 4. 評估與訓練流程 -----------
def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)  # L2
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping(patience=5)

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)  # L1
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 5. 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_incep = InceptionV3Transfer()
model_incep = train(model_incep, train_loader, val_loader, device)

test_acc = evaluate(model_incep, test_loader, device)
print(f"\n🎯 Inception-v3 Test Accuracy: {test_acc:.2f}%")


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1] Train Acc: 51.14% | Val Acc: 40.91%
[Epoch 2] Train Acc: 61.36% | Val Acc: 47.73%
[Epoch 3] Train Acc: 70.45% | Val Acc: 68.18%
[Epoch 4] Train Acc: 77.27% | Val Acc: 72.73%
[Epoch 5] Train Acc: 80.68% | Val Acc: 81.82%
[Epoch 6] Train Acc: 85.23% | Val Acc: 81.82%
[Epoch 7] Train Acc: 80.11% | Val Acc: 79.55%
[Epoch 8] Train Acc: 89.77% | Val Acc: 79.55%
[Epoch 9] Train Acc: 91.48% | Val Acc: 79.55%
[Epoch 10] Train Acc: 89.77% | Val Acc: 81.82%
⛔ Early stopping

🎯 Inception-v3 Test Accuracy: 76.25%


微調-解凍倒數3層

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 1. 資料前處理（299x299 for Inception） -----------
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=inception_transform)
test_dataset = datasets.ImageFolder(test_path, transform=inception_transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 2. 模型定義：Inception-v3 Transfer -----------
class InceptionV3Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.base = models.inception_v3(pretrained=True, aux_logits=True)

        # ✅ 凍結所有層
        for param in self.base.parameters():
            param.requires_grad = False

        # ✅ 解凍最後幾個 block（Mixed_7a, Mixed_7b, Mixed_7c）
        for name, param in self.base.named_parameters():
            if any(block in name for block in ["Mixed_7a", "Mixed_7b", "Mixed_7c"]):
                param.requires_grad = True

        self.base.fc = nn.Identity()  # 去掉原本分類層

        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.base(x)
        if isinstance(x, tuple):
            x = x.logits
        return self.classifier(x)



# ----------- 3. 損失計算與 EarlyStopping -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# ----------- 4. 評估與訓練流程 -----------
def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)  # L2
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping(patience=5)

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)  # L1
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 5. 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_incep = InceptionV3Transfer()
model_incep = train(model_incep, train_loader, val_loader, device)

test_acc = evaluate(model_incep, test_loader, device)
print(f"\n🎯 Inception-v3 Test Accuracy: {test_acc:.2f}%")


[Epoch 1] Train Acc: 53.98% | Val Acc: 59.09%
[Epoch 2] Train Acc: 91.48% | Val Acc: 81.82%
[Epoch 3] Train Acc: 95.45% | Val Acc: 88.64%
[Epoch 4] Train Acc: 99.43% | Val Acc: 93.18%
[Epoch 5] Train Acc: 100.00% | Val Acc: 90.91%
[Epoch 6] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 7] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 8] Train Acc: 100.00% | Val Acc: 90.91%
[Epoch 9] Train Acc: 100.00% | Val Acc: 93.18%
⛔ Early stopping

🎯 Inception-v3 Test Accuracy: 86.25%


微調-解凍後六層

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 1. 資料前處理（299x299 for Inception） -----------
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=inception_transform)
test_dataset = datasets.ImageFolder(test_path, transform=inception_transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 2. 模型定義：Inception-v3 Transfer -----------
class InceptionV3Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.base = models.inception_v3(pretrained=True, aux_logits=True)

        # ✅ 解凍後六層
        for name, param in self.base.named_parameters():
            if any(block in name for block in [
                "Mixed_6c", "Mixed_6d", "Mixed_6e",
                "Mixed_7a", "Mixed_7b", "Mixed_7c"
            ]):
                param.requires_grad = True
            else:
                param.requires_grad = False

        self.base.fc = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.base(x)
        if isinstance(x, tuple):
            x = x.logits
        return self.classifier(x)




# ----------- 3. 損失計算與 EarlyStopping -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# ----------- 4. 評估與訓練流程 -----------
def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)  # L2
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping(patience=5)

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)  # L1
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 5. 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_incep = InceptionV3Transfer()
model_incep = train(model_incep, train_loader, val_loader, device)

test_acc = evaluate(model_incep, test_loader, device)
print(f"\n🎯 Inception-v3 Test Accuracy: {test_acc:.2f}%")


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1] Train Acc: 55.11% | Val Acc: 70.45%
[Epoch 2] Train Acc: 85.23% | Val Acc: 75.00%
[Epoch 3] Train Acc: 94.89% | Val Acc: 79.55%
[Epoch 4] Train Acc: 98.86% | Val Acc: 84.09%
[Epoch 5] Train Acc: 99.43% | Val Acc: 84.09%
[Epoch 6] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 7] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 8] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 9] Train Acc: 100.00% | Val Acc: 81.82%
[Epoch 10] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 11] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 12] Train Acc: 100.00% | Val Acc: 88.64%
[Epoch 13] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 14] Train Acc: 99.43% | Val Acc: 84.09%
[Epoch 15] Train Acc: 99.43% | Val Acc: 84.09%
[Epoch 16] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 17] Train Acc: 100.00% | Val Acc: 81.82%
⛔ Early stopping

🎯 Inception-v3 Test Accuracy: 88.75%


微調-解凍後九層

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import copy

# ----------- 1. 資料前處理（299x299 for Inception） -----------
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\train"
test_path = r"C:\Users\PC\Desktop\學習歷程\雲科\學科\113-2\機器學習\作業\二\Gender classification\test"

dataset = datasets.ImageFolder(train_path, transform=inception_transform)
test_dataset = datasets.ImageFolder(test_path, transform=inception_transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------- 2. 模型定義：Inception-v3 Transfer -----------
class InceptionV3Transfer(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.base = models.inception_v3(pretrained=True, aux_logits=True)

        # ✅ 解凍後 9 層（Mixed_5d ~ Mixed_7c）
        for name, param in self.base.named_parameters():
            if any(block in name for block in [
                "Mixed_5d", "Mixed_6a", "Mixed_6b", "Mixed_6c",
                "Mixed_6d", "Mixed_6e", "Mixed_7a", "Mixed_7b", "Mixed_7c"
            ]):
                param.requires_grad = True
            else:
                param.requires_grad = False

        self.base.fc = nn.Identity()  # 移除原始分類層

        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.base(x)
        if isinstance(x, tuple):  # 處理 InceptionOutputs
            x = x.logits
        return self.classifier(x)


# ----------- 3. 損失計算與 EarlyStopping -----------
def compute_l1(model):
    return sum(torch.sum(torch.abs(p)) for p in model.parameters() if p.requires_grad)

class EarlyStopping:
    def __init__(self, patience=5):
        self.best_acc = 0
        self.counter = 0
        self.patience = patience
        self.best_model = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# ----------- 4. 評估與訓練流程 -----------
def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

def train(model, train_loader, val_loader, device, epochs=30):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-2)  # L2
    criterion = nn.CrossEntropyLoss()
    early = EarlyStopping(patience=5)

    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y) + 1e-4 * compute_l1(model)  # L1
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        val_acc = evaluate(model, val_loader, device)
        print(f"[Epoch {epoch+1}] Train Acc: {100*correct/total:.2f}% | Val Acc: {val_acc:.2f}%")

        if early.step(val_acc, model):
            print("⛔ Early stopping")
            break

    model.load_state_dict(early.best_model)
    return model

# ----------- 5. 執行訓練與測試 -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_incep = InceptionV3Transfer()
model_incep = train(model_incep, train_loader, val_loader, device)

test_acc = evaluate(model_incep, test_loader, device)
print(f"\n🎯 Inception-v3 Test Accuracy: {test_acc:.2f}%")


[Epoch 1] Train Acc: 54.55% | Val Acc: 72.73%
[Epoch 2] Train Acc: 86.93% | Val Acc: 81.82%
[Epoch 3] Train Acc: 96.02% | Val Acc: 84.09%
[Epoch 4] Train Acc: 100.00% | Val Acc: 81.82%
[Epoch 5] Train Acc: 100.00% | Val Acc: 79.55%
[Epoch 6] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 7] Train Acc: 100.00% | Val Acc: 86.36%
[Epoch 8] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 9] Train Acc: 99.43% | Val Acc: 88.64%
[Epoch 10] Train Acc: 100.00% | Val Acc: 84.09%
[Epoch 11] Train Acc: 99.43% | Val Acc: 84.09%
[Epoch 12] Train Acc: 100.00% | Val Acc: 77.27%
[Epoch 13] Train Acc: 99.43% | Val Acc: 88.64%
[Epoch 14] Train Acc: 100.00% | Val Acc: 88.64%
⛔ Early stopping

🎯 Inception-v3 Test Accuracy: 85.00%
